In [1]:
import pandas as pd 
import numpy as np

In [42]:
class network:
    def __init__(self):
        self.network = pd.DataFrame(columns=['node1', 'node2'])
        self.nodeInfo = pd.DataFrame(columns=['nodeId','p','c'])                       
    
    def add_node (self, info):
        if info[0] in self.nodeInfo.nodeId:
            print (f'could not add node {info[0]}, node allready in network')
            return
        self.nodeInfo.loc[len(self.nodeInfo)] = info
    def add_connection(self, connection):
        for i in connection:
            if i not in self.nodeInfo['nodeId'].to_list():
                print(f'could not add connection, you need to add node {i} first')
                return
        self.network.loc[len(self.network)] = connection

    def update_node_c (self,node,c):
        index = self.nodeInfo.query(f'nodeId =={node}').index[0]
        self.nodeInfo.at[index,'c'] = c

    def return_network(self):
        return self.network
    
    def find_all_conection_of_node(self, node):
        filterNode1 = self.network.query(f'node1 == {node}').node2
        filterNode2 = self.network.query(f'node2 == {node}').node1
        combinNodes = list(set(filterNode1.to_list()+filterNode2.to_list()))
        return combinNodes
    
    def check_for_path(self, node):
        allPossibleConections = []
        testedNodes =[]
        allPossibleConections = self.find_all_conection_of_node(node)
        for connectionNode in allPossibleConections:
            nextConnections = self.find_all_conection_of_node(connectionNode)
            for nc in nextConnections:
                if nc not in allPossibleConections:
                    allPossibleConections.append(nc)            
        return allPossibleConections
    
    def test_if_c_is_1(self,node):
        result = self.nodeInfo.query(f'nodeId =={node}').c 
        if not result.empty and result.iloc[0]==1:
            return True
        else:
            return False
    
    def get_mean_c_of_node(self, node):
        nodeList = self.find_all_conection_of_node(node)
        cList = []
        for node in nodeList:
            cList.append(self.nodeInfo.query(f'nodeId == {node}').c.to_numpy()[0])
        meanOfC = np.mean(cList)
        print(f'node {node} has mean {meanOfC}')
        return(meanOfC)
    
    
    def get_p_of_node(self,node):
        result = self.nodeInfo.loc[self.nodeInfo['nodeId'] == node, 'p']
        if result.empty:
                raise ValueError(f"No matching node found for nodeId: {node}")
            
        return result.iloc[0]
        #return self.nodeInfo.query(f'nodeId == {node}').p.to_numpy()[0]
    
    def check_cascade(self):
        ## ist noch broken
        nodesToCheck = self.nodeInfo.query('c == 1').nodeId.to_list()
        for singleNodeToCheck in nodesToCheck:
            connectionsOfNode = self.find_all_conection_of_node(singleNodeToCheck)
            for singleConnectionNode in connectionsOfNode:
                ## if node allready c=1
                if self.test_if_c_is_1(singleConnectionNode) == True:
                    break
                pOfNode = self.get_p_of_node(singleConnectionNode)
                meanCOfConections = self.get_mean_c_of_node(singleConnectionNode)
                if meanCOfConections>pOfNode:
                    self.update_node_c(singleConnectionNode,1)
                    


                    
                


            




In [51]:

n = network()
n.add_node([1,0.1,0])
n.add_node([2,0.9,1])
n.add_node([3,0.9,0])
n.add_node([4,0.9,0])
n.add_node([9,0.5,0])
n.add_node([8,0.5,0])
n.add_connection([1,2])
n.add_connection([1,3])
n.add_connection([4,2])
n.add_connection([3,2])
n.add_connection([9,8])

test = n.return_network()
n.nodeInfo

,nodeId,p,c
0,1.0,0.1,0.0
1,2.0,0.9,1.0
2,3.0,0.9,0.0
3,4.0,0.9,0.0
4,9.0,0.5,0.0
5,8.0,0.5,0.0


In [52]:
n.test_if_c_is_1(1)

False

In [53]:
n.check_cascade()
n.nodeInfo

,nodeId,p,c
0,1.0,0.1,1.0
1,2.0,0.9,1.0
2,3.0,0.9,1.0
3,4.0,0.9,1.0
4,9.0,0.5,0.0
5,8.0,0.5,0.0


In [54]:
n.nodeInfo.loc[n.nodeInfo['nodeId'] == 1, 'c'].iloc[0] == 19

False

In [55]:
testdf = n.nodeInfo

In [39]:
n.nodeInfo.query('c == 0').nodeId.to_list()

[1.0, 2.0, 3.0, 4.0, 9.0, 8.0]

In [48]:
nodeList =[1,2]
cList = []
for node in nodeList:
    cList.append(n.nodeInfo.query(f'nodeId == {node}').c.to_numpy()[0])
cList

[0.0, 0.0]

In [27]:
n.nodeInfo['nodeId']

0    1.0
1    2.0
2    3.0
3    4.0
4    9.0
5    8.0
Name: nodeId, dtype: float64

In [8]:
test_data = [1,2,0.5,0]
network = pd.DataFrame(columns=['node1', 'node2', 'p','c'])
network.loc[len(network)] = test_data
network

/tmp/ipykernel_6188/650769367.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([(pd.DataFrame([test_data],columns=network.columns)),network])


,node1,node2,p,c
0,1,2,0.5,0


In [5]:
index = testdf.query('nodeId ==2').index[0]
testdf.at[index,'c'] = 8
testdf

,nodeId,p,c
0,1.0,0.5,0.0
1,2.0,0.5,8.0
2,3.0,0.5,0.0
3,4.0,0.5,0.0
4,9.0,0.5,0.0
5,8.0,0.5,0.0
